## 补充




In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector # 导入projecter的包
# 载入数据集
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True) 
# x相对路径  数据集下载到某一目录下（相对路径不懂可自行百度）
# one-hot=True 把标签设置为one-hot 格式

Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz


In [14]:
# 运行次数
max_steps = 1001
# 图片数量
image_num = 3000
# 文件路径，
DIR = 'F:/Github/TensorFlow/深度学习框架Tensorflow学习与应用/'

# 定义会话
sess = tf.Session()

In [15]:
# 载入图片，把一定数量的测试图片载入内存
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
# 命名空间
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input', image_shaped_input,10) # 放进去10张图片

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
# 产生的metadata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv') #如果该文件存在，先删除
with open(DIR + 'projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))  # 拿到测试集所有标签，写入一部分到文件中去 
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')


# 合并所有summary
merged = tf.summary.merge_all()

# 定义一个writer，写入图
projector_writer = tf.summary.FileWriter(DIR + 'projector/projector',sess.graph)
#用来保存网络模型
saver = tf.train.Saver()
# 定义配置文件
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
# 切分图片
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer, config)


for i in range(max_steps):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_  = sess.run([merged, train_step], feed_dict={x:batch_xs,y:batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
        
saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'Input_3/y-input' with dtype float and shape [?,10]
	 [[node Input_3/y-input (defined at <ipython-input-13-bf71bf428aab>:18) ]]

Original stack trace for 'Input_3/y-input':
  File "C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\hasee\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\hasee\AppData\Loc